In [ ]:
# 라즈베리파이 -> 아두이노 값 전송 코드

# import serial
# import time

# com=serial.Serial(port="/dev/ttyACM0",
#                  baudrate=9600,
#                  bytesize=serial.EIGHTBITS,
#                  parity=serial.PARITY_NONE,
#                  timeout=1)
# while(True):
#     s='t'
#     com.write(s.encode())
#     time.sleep(5)
#     s='f'
#     com.write(s.encode())
#     time.sleep(1)
    

In [1]:
# 필요한 라이브러리 가져오기
import numpy as np
import imutils
import time
import timeit
import dlib
import cv2
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
from threading import Timer
from twilio.rest import Client

In [2]:
import pandas as pd
import webbrowser
data=pd.read_csv("C:/Users/ted33/OneDrive/문서/카카오톡 받은 파일/전국졸음쉼터표준데이터.csv", encoding='cp949')
ndata=data.drop(['도로노선번호','소재지도로명주소','총연장','방범용CCTV수','기타편의시설', '데이터기준일자','제공기관코드', '제공기관명' ], axis=1)

import folium as f
from folium.plugins import MarkerCluster
from scipy.spatial import distance
import math

In [4]:
# 눈의 가로세로 비율(EAR) 계산 함수
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# 주어진 프레임에서 조명을 제거하는 함수
def light_removing(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    L = lab[:, :, 0]
    med_L = cv2.medianBlur(L, 99)  # 노이즈를 줄이기 위한 중간값 필터 적용
    invert_L = cv2.bitwise_not(med_L)  # 밝기 반전
    composed = cv2.addWeighted(gray, 0.75, invert_L, 0.25, 0)  # 원본과 반전된 이미지 결합
    return L, composed

# 초기 눈을 뜬 상태의 EAR을 설정하는 함수
def init_open_ear():
    time.sleep(5)  # 5초 동안 눈을 뜬 상태를 기다립니다.
    print("눈을 뜬 EAR값을 저장합니다.")
    ear_list = []
    for i in range(7):
        ear_list.append(both_ear)
        time.sleep(1)
    global OPEN_EAR
    OPEN_EAR = sum(ear_list) / len(ear_list)

# 눈을 감은 상태의 EAR을 설정하는 함수
def init_close_ear():
    time.sleep(2)  # 2초 동안 눈을 감은 상태를 기다립니다.
    th_open.join()  # 눈을 뜬 상태의 초기화 스레드가 끝날 때까지 대기합니다.
    time.sleep(5)  # 5초 동안 눈을 감은 상태를 기다립니다.
    print("눈을 감은 EAR값을 저장합니다.")
    ear_list = []
    time.sleep(1)
    for i in range(7):
        ear_list.append(both_ear)
        time.sleep(1)
    CLOSE_EAR = sum(ear_list) / len(ear_list)
    global EAR_THRESH
    EAR_THRESH = (((OPEN_EAR - CLOSE_EAR) / 2) + CLOSE_EAR)
    print("마지막 EAR_THRESH 값: ", EAR_THRESH, "\n")
    
# 두 지점의 거리를 유클리디안 계산을 하는 함수    
def euclidean_distance(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)


# 가장 가까운 위치 3곳을 찾는 함수
def find_nearest_locations(current_location, data, num_locations=3):
    distances = {}
    for _, row in data.iterrows():
        location = (row['위도'], row['경도'])
        #dist1 = distance.euclidean(current_location, location)
        dist1=euclidean_distance(current_location,location)
        distances[(row['졸음쉼터명'], row['시도명'])] = dist1

    # 거리에 따라 정렬
    sorted_distances = sorted(distances.items(), key=lambda x: x[1])

    # 가장 가까운 3곳 선택
    nearest_locations = sorted_distances[:num_locations]

    return nearest_locations

#1. 초기화 변수 설정
OPEN_EAR = 0
EAR_THRESH = 0

#2. 눈 감음 감지 관련 변수 설정
EAR_CONSEC_FRAMES = 20
COUNTER = 0

#3. 타이머 및 알람 변수 설정
closed_eyes_time = []
TIMER_FLAG = False
ALARM_FLAG = False

#4. 알람 카운트 및 운영 시간 변수 설정
ALARM_COUNT = 0
RUNNING_TIME = 0

#5. 이전 시간 변수 설정
PREV_TERM = 0

#6. 훈련 데이터 생성
np.random.seed(9)

#7. Facial landmark detector 설정
print("Facial landmark predictor 로딩 중...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

#8. 비디오 스트림 시작
print("비디오 스트림 스레드 시작 중...")
vs = VideoStream(src=0).start()
time.sleep(1.0)

#9. 초기화 스레드 시작
th_open = Thread(target=init_open_ear)
th_open.daemon = True
th_open.start()
th_close = Thread(target=init_close_ear)
th_close.daemon = True
th_close.start()


while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=400)
    
    # 10. 조명 보정을 위해 `light_removing` 함수를 사용하여 프레임을 처리
    L, gray = light_removing(frame)
    
    # 11. 얼굴 감지 및 랜드마크 추출
    rects = detector(frame, 0)

    for rect in rects:
        shape = predictor(frame, rect)
        shape = face_utils.shape_to_np(shape)

        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        both_ear = (leftEAR + rightEAR) * 500
        
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

        if both_ear < EAR_THRESH:
            if not TIMER_FLAG:
                start_closing = timeit.default_timer()
                TIMER_FLAG = True
            COUNTER += 1

            if COUNTER >= EAR_CONSEC_FRAMES:
                mid_closing = timeit.default_timer()
                closing_time = round((mid_closing - start_closing), 3)

                if closing_time >= RUNNING_TIME:
                    if RUNNING_TIME == 0:
                        CUR_TERM = timeit.default_timer()
                        OPENED_EYES_TIME = round((CUR_TERM - PREV_TERM), 3)
                        PREV_TERM = CUR_TERM
                        RUNNING_TIME = 8

                    RUNNING_TIME += 2
                    ALARM_FLAG = True
                    ALARM_COUNT += 1

                    print("t")  # 졸음감지시 t 출력
                    # 지도 생성 및 시작 지점 설정
                    a=35.1401
                    b=126.9343
                    m = f.Map(location=[a,b], tiles='OpenStreetMap', zoom_start=11)

                    # 시작 지점 아이콘 추가
                    f.Marker(
                                location=[a,b],
                                icon=f.Icon(color='red'),
                                popup="현재 위치"
                    ).add_to(m)

                    # MarkerCluster 초기화
                    mc = MarkerCluster()



                    # 데이터 포인트를 MarkerCluster에 추가
                    for _, row in ndata.iterrows():
                        popup_content = f'<b>졸음쉼터명:</b> {row["졸음쉼터명"]} <b>시도명:</b> {row["시도명"]}<b>도로노선방향:</b> {row["도로노선방향"]}<b>소재지지번주소:</b> {row["소재지지번주소"]} <b>주차면수:</b> {row["주차면수"]} <b>화장실유무:</b> {row["화장실유무"]}'
                        mc.add_child(
                                        f.Marker(
                                                    location=[row['위도'], row['경도']],
                                                    popup=f.Popup(popup_content, parse_html=True)
                                                )
                                    )


                    # MarkerCluster를 지도에 추가
                    m.add_child(mc)

                    # LatLngPopup 추가
                    m.add_child(f.LatLngPopup())

                    # 현재 위치 (35.1401,126.9343)를 사용하여 가장 가까운 3곳 찾기
                    current_location = (a,b)
                    nearest_locations = find_nearest_locations(current_location, ndata, num_locations=3)

                    # 가장 가까운 3곳을 지도에 표시
                    for i, (location, distance) in enumerate(nearest_locations, start=1):
                            f.Marker(
                                    location=(ndata[ndata['졸음쉼터명'] == location[0]]['위도'].values[0], 
                                                  ndata[ndata['졸음쉼터명'] == location[0]]['경도'].values[0]),
                                    icon=f.Icon(color='green'),
                                    popup=f"추천 위치 {i}: {location[0]}, {location[1]} "
                            ).add_to(m)


                    # 지도 저장    
                    m.save("map.html")    
                    
                    # 지도 표시
                    webbrowser.open_new("map.html")
                    
                    # 졸음감지 문자 전송
                    account_sid='AC9560b4e8c221305504ce3e5cd5225f48'
                    auth_token=auth_token = 'afd1376e83faede053e1e019fc094b3a'
                    client = Client(account_sid, auth_token)
                    
                    message = client.messages.create(
                        to='821086206921',
                        from_='12563684985',
                        body = '운전자의 졸음을 감지 하였습니다'
                    )
                    
                    

        else:
            COUNTER = 0
            TIMER_FLAG = False
            RUNNING_TIME = 0

            if ALARM_FLAG:
                print("f")    # 졸음이 아닌 정상 상태 f 출력

            ALARM_FLAG = False
         
        # 캠 창에서 EAR 값을 실시간 출력
        cv2.putText(frame, "EAR : {:.2f}".format(both_ear), (300, 130), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 30, 20), 2)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break
        
        
cv2.destroyAllWindows()
vs.stop()


Facial landmark predictor 로딩 중...
비디오 스트림 스레드 시작 중...
눈을 뜬 EAR값을 저장합니다.
눈을 감은 EAR값을 저장합니다.
마지막 EAR_THRESH 값:  208.6415526880791 

t
f
t
f
